# Tutorial: Valve Unit Model with IAPWS Property Package

![](valve_1.svg)

## Learning Outcomes

- Demonstrate use of the valve unit model in IDAES


## Problem Statement

In this example, we will be passing a liquid stream through a valve.

**Stream Inputs:**

Flow Rate = 1000 mol/s

Temperature = 298 K

Inlet Pressure = 202650 Pa

Outlet Pressure = 101325 Pa

Valve Opening = 0.5 (fraction)

For more details, please refer to the IDAES documentation: https://idaes-pse.readthedocs.io/en/stable

## Setting up the problem in IDAES

In the following cell, we will be importing the necessary components from Pyomo and IDAES.

In [1]:
# Import math functions
import math

# Import objects from pyomo package 
from pyomo.environ import ConcreteModel, Constraint, value, SolverFactory, units

# Import the solver
from idaes.core.solvers import get_solver

# Import the main FlowsheetBlock from IDAES. The flowsheet block will contain the unit model
from idaes.core import FlowsheetBlock

# Import the Valve unit model
from idaes.models.unit_models import Valve

# Import the option to set the valve function
from idaes.models.unit_models import ValveFunctionType

# Import idaes logger to set output levels
import idaes.logger as idaeslog

# Import the iapws95 property package to create a property block for the flowsheet
from idaes.models.properties import iapws95

# Import the degrees_of_freedom function from the idaes.core.util.model_statistics package
# DOF = Number of Model Variables - Number of Model Constraints
from idaes.core.util.model_statistics import degrees_of_freedom

# Create the ConcreteModel and the FlowsheetBlock objects, and attach the flowsheet block to it.
m = ConcreteModel()

m.fs = FlowsheetBlock(dynamic=False) # dynamic or ss flowsheet needs to be specified here

# Add properties parameter block to the flowsheet
m.fs.properties = iapws95.Iapws95ParameterBlock()

In the following cell, we will be creating the valve unit model, assigning the valve function and property package to it, and determining the degrees of freedom associated with the valve unit model.

In [2]:
# Create an instance of the valve unit, attaching it to the flowsheet
# Specify which valve function should be used to describe how the flow coefficient is calculated
# Specify that the property package to be used with the valve is the one we created earlier

m.fs.valve = Valve(
    valve_function_callback=ValveFunctionType.linear,
    property_package=m.fs.properties
)

# Call the degrees_of_freedom function, get initial DOF
DOF_initial = degrees_of_freedom(m)
print('The initial degrees of freedom is: {0}'.format(DOF_initial))

The initial degrees of freedom is: 3


In [3]:
assert DOF_initial == 3

### Fixing input specifications
In the following cell, we will be specifying the inlet conditions for the valve block.

In [4]:
# Assign the known inlet conditions to variables
fin = 1000  # mol/s
pin = 202650  # Pa
pout = 101325  # Pa
tin = 298  # K

# Determine inlet enthalpy
hin = iapws95.htpx(T=tin * units.K, P=pin * units.Pa)  # J/mol

# Determine flow coefficient - equation depends on the valve function specified earlier
cv = 1000 / math.sqrt(pin - pout) / 0.5

# Fix the inlet conditions
m.fs.valve.inlet.enth_mol[0].fix(hin)
m.fs.valve.inlet.flow_mol[0].fix(fin)
m.fs.valve.inlet.pressure[0].fix(pin)
m.fs.valve.outlet.pressure[0].set_value(pout) # Sets the target value for the outlet pressure, but does not fix it
m.fs.valve.Cv.fix(cv)
m.fs.valve.valve_opening.fix(0.5)

# Call the degrees_of_freedom function, get final DOF
DOF_final = degrees_of_freedom(m)
print('The final degrees of freedom is: {0}'.format(DOF_final))

The final degrees of freedom is: 0


In [5]:
assert DOF_final == 0

### Flowsheet Initialization

In [6]:
# Initialize the flowsheet, and set the output at WARNING
m.fs.valve.initialize(outlvl=idaeslog.WARNING)

### Obtaining Simulation Results

In [7]:
# Solve the simulation using the IDAES solver
# Note: If the degrees of freedom = 0, we have a square problem
solver = get_solver()
result = solver.solve(m, tee=True)

Ipopt 3.13.2: nlp_scaling_method=gradient-based
tol=1e-06


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale

In [8]:
from pyomo.opt import TerminationCondition, SolverStatus

# Check if termination condition is optimal
assert result.solver.termination_condition == TerminationCondition.optimal
assert result.solver.status == SolverStatus.ok

### View Results

As expected, the pressure of the stream will be halved after going through the valve.

In [9]:
# Display output report
m.fs.valve.report()


Unit : fs.valve                                                            Time: 0.0
------------------------------------------------------------------------------------
    Unit Performance

    Variables: 

    Key               : Value       : Units                                 : Fixed : Bounds
      Mechanical Work :      0.0000 :                                  watt : False : (None, None)
              Opening :     0.50000 :                         dimensionless :  True : (None, None)
      Pressure Change : -1.0132e+05 :                                pascal : False : (None, None)
       Pressure Ratio :     0.50000 :                         dimensionless : False : (None, None)
    Valve Coefficient :      6.2831 : meter ** 0.5 * mole / kilogram ** 0.5 :  True : (None, None)

------------------------------------------------------------------------------------
    Stream Table
                                     Units           Inlet     Outlet  
    Molar Flow (mol/s)     

In [10]:
import pytest

# Check results
assert value(m.fs.valve.outlet.flow_mol[0]) == pytest.approx(1000, rel=1e-6)
assert value(m.fs.valve.outlet.pressure[0]) == pytest.approx(101325, rel=1e-6)
assert value(m.fs.valve.outlet.enth_mol[0]) == pytest.approx(1880.557, rel=1e-3)